In [12]:
from mongodb.dbs import CreateDropDbs
from mongodb.context_manager import MongoContextManager
from pprint import pprint

In [10]:
URI = "mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000"

<h1 style='text-align:center;'> Create, show a Database</h1>

In [14]:
with MongoContextManager(URI) as conn:
    dbs = CreateDropDbs(conn=conn)
    # dbs.create_dbs(db_name='unruffled_feather')


<h1 style='text-align:center;'> Getting a collection </h1>

<h1 style='text-align:center;'> Insert One </h1>

<h1 style='text-align:center;'> Insert Many </h1>

<h1 style='text-align:center;'> Update One </h1>

<h1 style='text-align:center;'> Update Many </h1>

<h1 style='text-align:center;'> Delete One and Many </h1>

<h1 style='text-align:center;'> Find </h1>

<h1 style='text-align:center;'> Find and Sort</h1>

<h1 style='text-align:center;'> Find and Limit</h1>

<h1 style='text-align:center;'> Find, Sort and Limit</h1>

<h1 style='text-align:center;'> Logical Operator</h1>

<p style="margin-bottom:20px; background-color=yellow;">not equal = $ne </p>
<p style="margin-bottom:20px; background-color=yellow;">exit = $exist </p>
<p style="margin-bottom:20px; background-color=yellow;">less than = $lt </p>
<p style="margin-bottom:20px; background-color=yellow;">less than or equal = $lte </p>
<p style="margin-bottom:20px; background-color=yellow;">greater than = $gt </p>
<p style="margin-bottom:20px; background-color=yellow;">greater than or equal = $gte </p>
<p style="margin-bottom:20px; background-color=yellow;">in value = $in </p>
<p style="margin-bottom:20px; background-color=yellow;">not in value = $nin </p>